In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import Normalizer, StandardScaler
from scipy.io import arff
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
import h5py
#from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)

In [ ]:
#load training dataset
dataTrain = arff.loadarff('/content/drive/My Drive/KDDTrain20MultiClass.arff')
traindata = pd.DataFrame(dataTrain[0])
dataTest = arff.loadarff('/content/drive/My Drive/KDDTest21MultiClass.arff')
testdata = pd.DataFrame(dataTrain[0])

In [ ]:
testdata.head

<bound method NDFrame.head of        duration  src_bytes  dst_bytes  land  ...  protocol_type  service  flag  xAttack
0           0.0      491.0        0.0  b'0'  ...           b'1'    b'20'  b'2'     b'5'
1           0.0      146.0        0.0  b'0'  ...           b'3'    b'45'  b'2'     b'5'
2           0.0        0.0        0.0  b'0'  ...           b'1'    b'50'  b'4'     b'1'
3           0.0      232.0     8153.0  b'0'  ...           b'1'    b'25'  b'2'     b'5'
4           0.0      199.0      420.0  b'0'  ...           b'1'    b'25'  b'2'     b'5'
...         ...        ...        ...   ...  ...            ...      ...   ...      ...
25187       0.0        0.0        0.0  b'0'  ...           b'1'    b'18'  b'3'     b'1'
25188       0.0      334.0        0.0  b'0'  ...           b'1'    b'20'  b'2'     b'3'
25189       0.0        0.0        0.0  b'0'  ...           b'1'    b'50'  b'1'     b'1'
25190       0.0        0.0        0.0  b'0'  ...           b'1'    b'42'  b'4'     b'1'
25

In [ ]:
X_train = traindata.iloc[:, :-1].values
Y_train = traindata.iloc[:, 41].values

X_test = testdata.iloc[:, :-1].values
Y_test = testdata.iloc[:, 41].values

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

from keras.utils.np_utils import to_categorical

y_train = np.array(Y_train)
y_test = np.array(Y_test)
y_train= to_categorical(y_train)
y_test= to_categorical(y_test)

In [ ]:
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)

In [ ]:
batch_size = 64

# 1. define the network
model = Sequential()
model.add(Dense(800,input_dim=41,activation='relu'))  
model.add(Dropout(0.9))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(6))
model.add(Activation('softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=1)
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Epoch 1/100
394/394 [==============================] - 3s 7ms/step - loss: 0.6222 - accuracy: 0.8523
Epoch 2/100
394/394 [==============================] - 3s 7ms/step - loss: 0.2868 - accuracy: 0.9356
Epoch 3/100
394/394 [==============================] - 3s 8ms/step - loss: 0.2358 - accuracy: 0.9456
Epoch 4/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1974 - accuracy: 0.9505
Epoch 5/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1887 - accuracy: 0.9568
Epoch 6/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1666 - accuracy: 0.9598
Epoch 7/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1585 - accuracy: 0.9630
Epoch 8/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1911 - accuracy: 0.9647
Epoch 9/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1858 - accuracy: 0.9661
Epoch 10/100
394/394 [==============================] - 3s 7ms/step - loss: 0.1747 - accura

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

target_names = ['DoS', 'U2R', 'R2L', 'Probe', 'Normal', 'Unknown']

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))


              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00         0
         U2R       1.00      1.00      1.00      9234
         R2L       0.00      0.00      0.00        11
       Probe       0.97      0.42      0.58       209
      Normal       0.99      0.96      0.97      2289
     Unknown       0.99      0.99      0.99     13449

   micro avg       0.99      0.99      0.99     25192
   macro avg       0.66      0.56      0.59     25192
weighted avg       0.99      0.99      0.99     25192
 samples avg       0.99      0.99      0.99     25192

Accuracy_Train: 0.9851540171483011
Precision_Train: 0.9907676619476649
Recall_Train: 0.9851540171483011
f1-score_Train: 0.9870371980266983


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
import sklearn.metrics as metrics

expected = y_test
predicted = model.predict(X_test)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
#classification report
print(classification_report(expected, np.round(predicted), target_names=target_names))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

Accuracy: 0.9851540171483011
Precision: 0.9907676619476649
Recall: 0.9851540171483011
f1-score: 0.9870371980266983
              precision    recall  f1-score   support

         DoS       0.00      0.00      0.00         0
         U2R       1.00      1.00      1.00      9234
         R2L       0.00      0.00      0.00        11
       Probe       0.97      0.42      0.58       209
      Normal       0.99      0.96      0.97      2289
     Unknown       0.99      0.99      0.99     13449

   micro avg       0.99      0.99      0.99     25192
   macro avg       0.66      0.56      0.59     25192
weighted avg       0.99      0.99      0.99     25192
 samples avg       0.99      0.99      0.99     25192



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

ValueError: ignored